# Simple AI Coding Agent Using The REST API (And Some Python)

## Python Setup

Imports some modules for HTTPS requests and handling JSON

In [1]:
import json
import os

import requests

Set the URL, Headers and API Key

In [66]:
URL = "https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent" 

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

headers = {
    "x-goog-api-key": GEMINI_API_KEY,
    "Content-Type": "application/json",
}

## Some Utility Functions

In [3]:
def print_answer(resp, debug=False):
    res = json.loads(resp.text)
    if debug:
        print(res)
        print()
    role = res['candidates'][0]['content']['role']
    answer = res['candidates'][0]['content']['parts'][0]['text']
    print(f"{role}: {answer}")

In [23]:
def make_request(payload):
    response = requests.post(URL, headers=headers, data=json.dumps(payload))
    if response.status_code == 200:
        print_answer(response)
    else:
        print(f"Errror, status code: {response.status_code} details: {response.text}")

def make_request_with_tools(payload, debug=False):
    response = requests.post(URL, headers=headers, data=json.dumps(payload))
    if response.status_code == 200:
        res = json.loads(response.text)
        if debug:
            print(res)
            print()
            parts = res['candidates'][0]['content']['parts'][0]
            role = res['candidates'][0]['content']['role']
            if 'functionCall' in parts:
                print(f"{Tool call}: {parts['functionCall']}")
            # answer = ['text']
            # print(f"{role}: {answer}")
    else:
        print(f"Errror, status code: {response.status_code} details: {response.text}")

## A Simple Chat Request

A simple call to an LLM for a response involves sending a JSON payload in a HTTPS POST request. 

The content of the "chat" is in the 'text' section of the 'parts'. The 'contents' array is an array of objects that make up the contents of the call to the LLM.

In [50]:
payload = {
    "contents": [
      {
        "role": "user",
        "parts": [
          {
            "text": "Explain how AI works in a few words"
          }
        ]
      }
    ]
}

make_request(payload)

model: It learns from data to find patterns and make smart decisions.


## Defining Model Behaviour With A System Instruction

In [53]:
system_instructions = {
    "system_instruction": {
      "parts": [
        {
          "text": "You are a software engineer. Your name is John's AI Coding Bot."
        }
      ]
    }
}

In [54]:
payload = system_instructions | {
    "contents": [
      {
        "role": "user",
        "parts": [
          {
            "text": "Who are you? What do you do?"
          }
        ]
      }
    ]
}

make_request(payload)

model: Hello! I am John's AI Coding Bot.

What I do is assist with a wide range of programming-related tasks. Think of me as a digital co-pilot for your coding journey. I can help you with:

*   **Writing code:** Generating code snippets, functions, or even entire scripts in various languages.
*   **Debugging:** Helping you identify and fix errors in your code.
*   **Code review:** Providing feedback on code quality, readability, and best practices.
*   **Explaining concepts:** Clarifying programming concepts, algorithms, design patterns, or specific language features.
*   **Refactoring:** Suggesting ways to improve the structure and efficiency of your code.
*   **Designing architectures:** Discussing different approaches for system design and database schemas.
*   **Learning and tutorials:** Guiding you through new technologies or helping you understand complex topics.
*   **Answering questions:** Providing information on APIs, libraries, frameworks, and general software development k

## Add Some Tools

In [55]:
tools = {
"tools": [
        {
            "function_declarations": [
                {
                    "name": "write_code",
                    "description": "writes some code to a file",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "filename": {
                                "type": "string",
                                "description": "The filename to write the code to"
                            },
                            "code": {
                                "type": "string",
                                "description": "The code to write to the file"
                            }
                        },
                        "required": [
                            "filename",
                            "code"
                        ]
                    }
                }
            ]
        }
    ]
}

In [56]:
payload = system_instructions | tools | {    
    "contents": [
        {
            "role": "user",
            "parts": [
                {
                    "text": "Who are you? What do you do? What tools can you use?"
                }
            ]
        }
    ]
}

make_request(payload)

model: I am John's AI Coding Bot, a software engineer. I write code to files. I can use the `write_code` tool to write code to a specified filename.


## Multiturn Work

In [80]:
payload = system_instructions | tools | {    
    "contents": []
}

first_user_request = {
    "role": "user",
    "parts": [
        {
            "text": "Write a python script to output the first 10 numbers of the Fibonacci sequence."
        }
    ]
}

payload["contents"].append(first_user_request)

response = requests.post(URL, headers=headers, data=json.dumps(payload))
if response.status_code == 200:
    res = json.loads(response.text)
    print(res)
else:
    print(f"Errror, status code: {response.status_code} details: {response.text}")

# make_request_with_tools(payload, debug=True)

{'candidates': [{'content': {'parts': [{'functionCall': {'name': 'write_code', 'args': {'code': '\ndef fibonacci(n):\n    a, b = 0, 1\n    for i in range(n):\n        print(a)\n        a, b = b, a + b\n\nfibonacci(10)\n', 'filename': 'fibonacci.py'}}, 'thoughtSignature': 'Co4DAXLI2nz/mv5bUJLlPM7lzfXe1i34rVq3krLClz06+HK6ZCSoZTU9ugk0agkKJ67mRYQgbpYMvnK7Gv2IyUZmxwaHP+oX9zv9eoYeKA28zUR4e+pAWflTPlg8I7VIE5XYM8R7RzEaeG2MLAFPqubwETDvrSt7m6CLWuGRzSEMz4EAe54HzNc81qmOPV7wt9yFkOTMIuFtRWFYEli8KOiQ35Vj4pM9WMNENQBX2i21ByohX/fSfvgvVKzntaLuZR10oeEYou85yMxF8/LRKPpFfURaU/S+OEB3rvP5vQ1A+wi659ttfNno9TNQKvqtYW63IKSqifXolPBVpEvn0/U21jZJGT2u7il6q+oCI3zEhZ2psdm98IkL+++B/M8BZBWHqVzKVe1AcYtsvtruDMF1AwQ9GzKlLeU1YQXUt5wNmNJDsWaiJ5XAijjGrb2do2DQ5wG0TRWx+cBbd47au/5KLYf52SIXIVit9F1W2Gza1Udmqy2TraFQHwZhbTM9Ciyu2M2GmG7/lUsSUONFUbg='}], 'role': 'model'}, 'finishReason': 'STOP', 'index': 0, 'finishMessage': 'Model generated function call(s).'}], 'usageMetadata': {'promptTokenCount': 95, 'candidatesTokenCount': 77, 'total

In [81]:
tool_call = {}
parts = res['candidates'][0]['content']['parts'][0]
role = res['candidates'][0]['content']['role']
if 'functionCall' in parts:
    tool_call = res['candidates'][0]['content']
    function = parts['functionCall']['name']
    filename = parts['functionCall']['args']['filename']
    code = parts['functionCall']['args']['code']
    print(f"Tool call requested\nFunction: `{function}` with filename: `{filename}`")
    print("Provided code:")
    print(code)
if 'text' in parts:
    print(f"{role}: {parts['text']}")

Tool call requested
Function: `write_code` with filename: `fibonacci.py`
Provided code:

def fibonacci(n):
    a, b = 0, 1
    for i in range(n):
        print(a)
        a, b = b, a + b

fibonacci(10)



## Simulating A Response

For a real tool, we'd now call that function with the provide arguments and then send the model a tool response, like the following:

In [82]:
tool_response = {
    "role": "user",
    "parts": [{
        "functionResponse": {
            "name": "write_code",
            "response": {
                "success": True,
                "output": "File written successfully"
            }
        }
    }]
}

In [83]:
# Next we call the model, recording it's tool call
payload["contents"].append(tool_call)

# And the tool response as new messages in the contents list
payload["contents"].append(tool_response)

# Let's see the payload
print(json.dumps(payload, indent=4))


{
    "system_instruction": {
        "parts": [
            {
                "text": "You are a software engineer. Your name is John's AI Coding Bot."
            }
        ]
    },
    "tools": [
        {
            "function_declarations": [
                {
                    "name": "write_code",
                    "description": "writes some code to a file",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "filename": {
                                "type": "string",
                                "description": "The filename to write the code to"
                            },
                            "code": {
                                "type": "string",
                                "description": "The code to write to the file"
                            }
                        },
                        "required": [
                            "filename",
 

In [84]:
make_request(payload)

model: I have written the Python script to output the first 10 numbers of the Fibonacci sequence and saved it as `fibonacci.py`.
